In [ ]:
!rm -r '/content/qani'

In [ ]:
!zip -r predicted_vs_actual_plots.zip /content/predicted_vs_actual_plots

In [ ]:
import pandas as pd

# Step 1: Load the Excel file into a DataFrame
df = pd.read_excel('/content/drive/MyDrive/Qani_datas.xlsx')

In [ ]:
df1 = df.copy()

In [ ]:
df.columns

Index(['time', 'timestamp_received', 'timestamp', 'trip', 'signal', 'isLive',
       'Battery_voltage', 'Fuel_level', 'Intake_manifold_absolute_pressure',
       'Throttle_position', 'Accelerator_pedal_position',
       'Coolant_temperature', 'Vehicle_Speed', 'Engine_speed',
       'Target_air_fuel_ratio', 'Current_gear_shift_position_(Current_gear)',
       'Cumulative_mileage', 'Clutch_torque', 'Trip_fuel_consumption',
       'RON_factor', '0313', '0314', '033d', '0345', '0348', '0409', '040a',
       'altitude', 'latitude', 'longitude', 'satelites', 'bearing',
       'angular_speed'],
      dtype='object')

# **create in one file**

In [ ]:
# Step 2: Group by 'trip' column (replace 'trip' with your actual column name for trip)
grouped = df.groupby('trip')

# Step 3: Define a function to check if the time and trip_fuel_consumption columns are monotonically increasing
def check_monotonic(group):
    if group['time'].is_monotonic_increasing and group['Trip_fuel_consumption'].is_monotonic_increasing:
        return group  # Keep the group if both columns are monotonically increasing
    else:
        return None  # Return None if the criteria aren't met

# Step 4: Apply the function to each group, filter out None values, and concatenate the valid trips
valid_trips = pd.concat([group for group in [check_monotonic(group) for _, group in grouped] if group is not None])

# Step 5: Check if there are any valid trips before saving
if not valid_trips.empty:
    # Save the valid trips to a new Excel file if they exist
    valid_trips.to_excel('valid_trips.xlsx', index=False)
    print("Valid trips saved to 'valid_trips.xlsx'")
else:
    # If no valid trips exist, do nothing (return nothing)
    print("No valid trips found.")


KeyboardInterrupt: 

# **Create seprate file with check time monotonic**



In [ ]:

# Step 2: Group by 'trip' column (replace 'number_of_trip' with your actual column name)
grouped = df1.groupby('trip')
folder = 'qani'
# Step 3: Define a function to check for time jumps and save the valid portion
def check_monotonic_and_save(group, trip_id):
    # Check for monotonic increase in the 'time' column
    is_monotonic = group['time'].is_monotonic_increasing

    if is_monotonic:
        # If the 'time' column is completely monotonic, save the whole trip
        filename = f'qani/qani_{trip_id}_valid.csv'
        group.to_csv(filename, index=False)
        print(f'Saved full trip: {filename}')
    else:
        # If there is a jump, find where the time decreases or jumps
        diff = group['time'].diff()  # Get the difference between consecutive time values
        jump_index = diff[diff <= 0].index[0]  # Find the first index where time decreases or jumps

        # Save the portion before the jump
        valid_portion = group.loc[:jump_index - 1]  # Everything before the first jump

        filename = f'qani/qani_{trip_id}_partial.csv'
        valid_portion.to_csv(filename, index=False)
        print(f'Saved partial trip: {filename}')

# Step 4: Apply the function to each group and save them separately
for trip_id, group in grouped:
    check_monotonic_and_save(group, trip_id)



Saved full trip: qani/qani_1_valid.csv
Saved full trip: qani/qani_2_valid.csv
Saved full trip: qani/qani_3_valid.csv
Saved full trip: qani/qani_4_valid.csv
Saved full trip: qani/qani_5_valid.csv
Saved full trip: qani/qani_6_valid.csv
Saved full trip: qani/qani_7_valid.csv
Saved full trip: qani/qani_8_valid.csv
Saved full trip: qani/qani_9_valid.csv
Saved full trip: qani/qani_10_valid.csv
Saved full trip: qani/qani_11_valid.csv
Saved full trip: qani/qani_12_valid.csv
Saved partial trip: qani/qani_13_partial.csv
Saved partial trip: qani/qani_14_partial.csv
Saved full trip: qani/qani_15_valid.csv
Saved full trip: qani/qani_16_valid.csv
Saved full trip: qani/qani_17_valid.csv
Saved partial trip: qani/qani_18_partial.csv
Saved full trip: qani/qani_19_valid.csv
Saved partial trip: qani/qani_20_partial.csv
Saved full trip: qani/qani_21_valid.csv
Saved full trip: qani/qani_22_valid.csv
Saved partial trip: qani/qani_23_partial.csv
Saved full trip: qani/qani_24_valid.csv
Saved full trip: qani/qa

# **Check the fuel consumption**

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define the folder where CSV files are stored and the folder for saving plots
folder = 'qani'
output_folder = 'combined_plots'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Initialize an empty DataFrame to hold the combined data
combined_df = pd.DataFrame()

# Step 1: Loop through all CSV files in the folder and append the data
for filename in os.listdir(folder):
    if filename.endswith('.csv'):  # Process only CSV files
        file_path = os.path.join(folder, filename)
        df = pd.read_csv(file_path)

        # Combine only the numeric columns and ensure necessary columns exist
        if df.select_dtypes(include='number').shape[1] > 0:
            df['trip_id'] = filename  # Add a column to distinguish trips
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        else:
            print(f"File {filename} does not contain numeric columns.")
combined_df.to_csv('combined.csv', index=False)

In [ ]:
import os
import pandas as pd

# Define the folder path
folder = 'qani'

# Initialize a counter for the total number of rows
total_rows = 0

# Step 1: Loop through all files in the folder
for filename in os.listdir(folder):
    # Check if the file is a CSV file
    if filename.endswith('.csv'):
        # Read the CSV file
        file_path = os.path.join(folder, filename)
        df = pd.read_csv(file_path)

        # Count the number of rows and add to the total
        row_count = len(df)
        total_rows += row_count

        print(f'{filename} has {row_count} rows.')

# Step 2: Print the total number of rows across all CSV files
print(f'Total number of rows in all CSV files: {total_rows}')


qani_107_valid.csv has 21 rows.
qani_74_valid.csv has 8848 rows.
qani_96_valid.csv has 2088 rows.
qani_186_valid.csv has 2491 rows.
qani_141_valid.csv has 2560 rows.
qani_26_valid.csv has 1974 rows.
qani_148_valid.csv has 1676 rows.
qani_172_valid.csv has 1562 rows.
qani_43_valid.csv has 153 rows.
qani_31_valid.csv has 416 rows.
qani_52_valid.csv has 691 rows.
qani_194_valid.csv has 41 rows.
qani_120_valid.csv has 1136 rows.
qani_159_partial.csv has 448 rows.
qani_124_valid.csv has 2588 rows.
qani_131_valid.csv has 1146 rows.
qani_166_valid.csv has 3340 rows.
qani_29_valid.csv has 1 rows.
qani_198_valid.csv has 892 rows.
qani_93_valid.csv has 2792 rows.
qani_154_valid.csv has 428 rows.
qani_82_valid.csv has 740 rows.
qani_173_valid.csv has 1401 rows.
qani_179_valid.csv has 2674 rows.
qani_99_valid.csv has 820 rows.
qani_62_valid.csv has 2601 rows.
qani_76_partial.csv has 39 rows.
qani_157_valid.csv has 250 rows.
qani_7_valid.csv has 2914 rows.
qani_213_valid.csv has 132 rows.
qani_37_v

# **Save box plot for all colomn**

In [ ]:
import matplotlib.pyplot as plt
import os

output_folder = 'box_plots'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # Create folder if it doesn't exist

# Step 3: Create and save a box plot for each numeric column
for column in combined_df.select_dtypes(include='number').columns:
    plt.figure(figsize=(10, 6))  # Optional: adjust figure size
    combined_df.boxplot(column=column)  # Create a box plot for the current column
    plt.title(f'Box Plot for {column}')  # Title for the plot
    plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
    # Save the box plot to the specified folder
    boxplot_filename = os.path.join(output_folder, f'boxplot_{column}.png')
    plt.savefig(boxplot_filename)  # Save the box plot as a .png file
    plt.close()  # Close the plot

    print(f'Box plot saved as: {boxplot_filename}')

Box plot saved as: box_plots/boxplot_time.png
Box plot saved as: box_plots/boxplot_trip.png
Box plot saved as: box_plots/boxplot_signal.png
Box plot saved as: box_plots/boxplot_Battery_voltage.png
Box plot saved as: box_plots/boxplot_Fuel_level.png
Box plot saved as: box_plots/boxplot_Intake_manifold_absolute_pressure.png
Box plot saved as: box_plots/boxplot_Throttle_position.png


Exception ignored in: <function ZipFile.__del__ at 0x7825b621a8c0>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1834, in __del__
    self.close()
  File "/usr/lib/python3.10/zipfile.py", line 1851, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


Box plot saved as: box_plots/boxplot_Accelerator_pedal_position.png
Box plot saved as: box_plots/boxplot_Coolant_temperature.png
Box plot saved as: box_plots/boxplot_Vehicle_Speed.png
Box plot saved as: box_plots/boxplot_Engine_speed.png
Box plot saved as: box_plots/boxplot_Target_air_fuel_ratio.png
Box plot saved as: box_plots/boxplot_Current_gear_shift_position_(Current_gear).png
Box plot saved as: box_plots/boxplot_Cumulative_mileage.png
Box plot saved as: box_plots/boxplot_Clutch_torque.png
Box plot saved as: box_plots/boxplot_Trip_fuel_consumption.png
Box plot saved as: box_plots/boxplot_RON_factor.png
Box plot saved as: box_plots/boxplot_0313.png
Box plot saved as: box_plots/boxplot_0314.png
Box plot saved as: box_plots/boxplot_033d.png
Box plot saved as: box_plots/boxplot_0345.png
Box plot saved as: box_plots/boxplot_0348.png
Box plot saved as: box_plots/boxplot_0409.png
Box plot saved as: box_plots/boxplot_040a.png
Box plot saved as: box_plots/boxplot_altitude.png
Box plot save

# **Violin plot**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns  # Import seaborn for violin plots
import os

output_folder = 'violin_plots'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # Create folder if it doesn't exist

# Step 3: Create and save a violin plot for each numeric column
for column in combined_df.select_dtypes(include='number').columns:
    plt.figure(figsize=(10, 6))  # Optional: adjust figure size
    sns.violinplot(y=combined_df[column])  # Create a violin plot for the current column
    plt.title(f'Violin Plot for {column}')  # Title for the plot
    plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability (if needed)

    # Save the violin plot to the specified folder
    violinplot_filename = os.path.join(output_folder, f'violinplot_{column}.png')
    plt.savefig(violinplot_filename)  # Save the violin plot as a .png file
    plt.close()  # Close the plot

    print(f'Violin plot saved as: {violinplot_filename}')


/usr/local/lib/python3.10/dist-packages/seaborn/_base.py:949: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


Violin plot saved as: violin_plots/violinplot_time.png


/usr/local/lib/python3.10/dist-packages/seaborn/_base.py:949: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


Violin plot saved as: violin_plots/violinplot_trip.png


/usr/local/lib/python3.10/dist-packages/seaborn/_base.py:949: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


...

Violin plot saved as: violin_plots/violinplot_satelites.png


/usr/local/lib/python3.10/dist-packages/seaborn/_base.py:949: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


Violin plot saved as: violin_plots/violinplot_bearing.png


/usr/local/lib/python3.10/dist-packages/seaborn/_base.py:949: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


Violin plot saved as: violin_plots/violinplot_angular_speed.png


# **Create seprate file with check time and fuel consumption monotonic**



In [ ]:
!zip -r qani.zip qani

  adding: qani/ (stored 0%)
  adding: qani/qani_107_valid.csv (deflated 73%)
  adding: qani/qani_74_valid.csv (deflated 80%)
  adding: qani/qani_96_valid.csv (deflated 79%)
  adding: qani/qani_186_valid.csv (deflated 82%)
  adding: qani/qani_141_valid.csv (deflated 80%)
  adding: qani/qani_26_valid.csv (deflated 80%)
  adding: qani/qani_148_valid.csv (deflated 79%)
  adding: qani/qani_172_valid.csv (deflated 79%)
  adding: qani/qani_43_valid.csv (deflated 85%)
  adding: qani/qani_31_valid.csv (deflated 87%)
  adding: qani/qani_52_valid.csv (deflated 80%)
  adding: qani/qani_194_valid.csv (deflated 89%)
  adding: qani/qani_120_valid.csv (deflated 79%)
  adding: qani/qani_159_partial.csv (deflated 78%)
  adding: qani/qani_124_valid.csv (deflated 80%)
  adding: qani/qani_131_valid.csv (deflated 79%)
  adding: qani/qani_166_valid.csv (deflated 81%)
  adding: qani/qani_29_valid.csv (deflated 36%)
  adding: qani/qani_198_valid.csv (deflated 83%)
  adding: qani/qani_93_valid.csv (deflated 80%

In [ ]:
!zip -r qani_2.zip qani_2

In [ ]:
!zip -r box_plots.zip box_plots

  adding: box_plots/ (stored 0%)
  adding: box_plots/boxplot_0409.png (deflated 31%)
  adding: box_plots/boxplot_Trip_fuel_consumption.png (deflated 25%)
  adding: box_plots/boxplot_0314.png (deflated 32%)
  adding: box_plots/boxplot_0313.png (deflated 31%)
  adding: box_plots/boxplot_time.png (deflated 27%)
  adding: box_plots/boxplot_Battery_voltage.png (deflated 26%)
  adding: box_plots/boxplot_Engine_speed.png (deflated 26%)
  adding: box_plots/boxplot_Cumulative_mileage.png (deflated 27%)
  adding: box_plots/boxplot_longitude.png (deflated 27%)
  adding: box_plots/boxplot_Target_air_fuel_ratio.png (deflated 24%)
  adding: box_plots/boxplot_Accelerator_pedal_position.png (deflated 25%)
  adding: box_plots/boxplot_signal.png (deflated 27%)
  adding: box_plots/boxplot_Fuel_level.png (deflated 26%)
  adding: box_plots/boxplot_Current_gear_shift_position_(Current_gear).png (deflated 24%)
  adding: box_plots/boxplot_0345.png (deflated 32%)
  adding: box_plots/boxplot_latitude.png (defla

In [ ]:
!zip -r violin_plots.zip violin_plots

  adding: violin_plots/ (stored 0%)
  adding: violin_plots/violinplot_033d.png (deflated 33%)
  adding: violin_plots/violinplot_0409.png (deflated 33%)
  adding: violin_plots/violinplot_Coolant_temperature.png (deflated 13%)
  adding: violin_plots/violinplot_Trip_fuel_consumption.png (deflated 12%)
  adding: violin_plots/violinplot_Throttle_position.png (deflated 13%)
  adding: violin_plots/violinplot_040a.png (deflated 33%)
  adding: violin_plots/violinplot_Battery_voltage.png (deflated 19%)
  adding: violin_plots/violinplot_signal.png (deflated 10%)
  adding: violin_plots/violinplot_Intake_manifold_absolute_pressure.png (deflated 11%)
  adding: violin_plots/violinplot_Cumulative_mileage.png (deflated 14%)
  adding: violin_plots/violinplot_Fuel_level.png (deflated 9%)
  adding: violin_plots/violinplot_0314.png (deflated 33%)
  adding: violin_plots/violinplot_Accelerator_pedal_position.png (deflated 14%)
  adding: violin_plots/violinplot_0313.png (deflated 34%)
  adding: violin_plots/v

In [18]:
!zip -r valid_trips_csv.zip valid_trips_csv

  adding: valid_trips_csv/ (stored 0%)
  adding: valid_trips_csv/Trip_141.csv (deflated 82%)
  adding: valid_trips_csv/Trip_184.csv (deflated 83%)
  adding: valid_trips_csv/Trip_166.csv (deflated 83%)
  adding: valid_trips_csv/Trip_121.csv (deflated 88%)
  adding: valid_trips_csv/Trip_97.csv (deflated 82%)
  adding: valid_trips_csv/Trip_155.csv (deflated 82%)
  adding: valid_trips_csv/Trip_125.csv (deflated 82%)
  adding: valid_trips_csv/Trip_217.csv (deflated 82%)
  adding: valid_trips_csv/Trip_21.csv (deflated 82%)
  adding: valid_trips_csv/Trip_120.csv (deflated 82%)
  adding: valid_trips_csv/Trip_180.csv (deflated 82%)
  adding: valid_trips_csv/Trip_79.csv (deflated 82%)
  adding: valid_trips_csv/Trip_178.csv (deflated 89%)
  adding: valid_trips_csv/Trip_27.csv (deflated 82%)
  adding: valid_trips_csv/Trip_112.csv (deflated 82%)
  adding: valid_trips_csv/Trip_63.csv (deflated 85%)
  adding: valid_trips_csv/Trip_115.csv (deflated 83%)
  adding: valid_trips_csv/Trip_58.csv (deflated 

# **Altitude**

In [12]:
import pandas as pd
import os

# Path to the folder containing CSV files
folder_path = '/content/'

# Process each CSV file in the specified folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")

        # Read the CSV file
        data_df = pd.read_csv(file_path)

        # Sort the DataFrame by 'time' and 'trip'
        data_df.sort_values(by=['trip', 'time'], inplace=True)

        # Initialize the 'slope' column with 0.0
        data_df['slope'] = 0.0

        # Group by 'trip' and calculate slope for each group
        for trip, group in data_df.groupby('trip'):
            # Check if the group has at least 100 rows
            num_rows = len(group)
            for i in range(0, num_rows - 100, 100):
                start_altitude = group.iloc[i]['altitude']
                end_altitude = group.iloc[i + 100]['altitude']
                start_mileage = group.iloc[i]['Cumulative_mileage']
                end_mileage = group.iloc[i + 100]['Cumulative_mileage']

                if end_mileage - start_mileage != 0:
                    slope = (end_altitude - start_altitude) / (end_mileage - start_mileage) / 10
                    data_df.loc[group.index[i:i + 99], 'slope'] = slope

            # Calculate slope for remaining rows if less than 100 remain
            if num_rows % 100 != 0:
                last_block_start = num_rows - (num_rows % 100)
                if last_block_start > 0:
                    start_altitude = group.iloc[last_block_start - 1]['altitude']
                    end_altitude = group.iloc[num_rows - 1]['altitude']
                    start_mileage = group.iloc[last_block_start - 1]['Cumulative_mileage']
                    end_mileage = group.iloc[num_rows - 1]['Cumulative_mileage']

                    if end_mileage - start_mileage != 0:
                        slope = (end_altitude - start_altitude) / (end_mileage - start_mileage) / 10
                        data_df.loc[group.index[last_block_start - 1:num_rows - 1], 'slope'] = slope

        # Define output file path
        output_file_path = os.path.join(folder_path, f"{filename[:-4]}_slope_added.csv")

        # Save the updated DataFrame to a new CSV file
        data_df.to_csv(output_file_path, index=False)

print("All files processed successfully.")


Processing file: combined.csv
All files processed successfully.


# **Valid trip**

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configure plot aesthetics (optional)
sns.set(style="whitegrid")



def identify_valid_trips(data_df, trip_column='trip', time_column='time', min_interval=400, max_interval=2000,
                         min_rows=1000):
    """
    Identifies valid trips based on time interval conditions and minimum row count.

    Parameters:
        data_df (pd.DataFrame): DataFrame containing trip and time data.
        trip_column (str): Name of the column representing trip identifiers.
        time_column (str): Name of the column representing time entries.
        min_interval (int): Minimum allowed time interval between consecutive rows.
        max_interval (int): Maximum allowed time interval between consecutive rows.
        min_rows (int): Minimum number of rows required for a trip to be considered valid.

    Returns:
        List of valid trip identifiers.
    """
    # Ensure required columns exist
    if trip_column not in data_df.columns or time_column not in data_df.columns:
        raise ValueError(f"Data sheet must contain '{trip_column}' and '{time_column}' columns.")

    # Sort data by trip and time to ensure chronological order
    data_sorted = data_df.sort_values(by=[trip_column, time_column]).reset_index(drop=True)

    # Group by trip and calculate time differences
    data_sorted['Time_Diff'] = data_sorted.groupby(trip_column)[time_column].diff()

    # Calculate the number of rows per trip
    trip_counts = data_sorted.groupby(trip_column).size().reset_index(name='Row_Count')

    # Identify trips where all time differences are within the specified range
    # and the number of rows is greater than min_rows
    trip_validity = data_sorted.groupby(trip_column).agg({
        'Time_Diff': lambda x: x.dropna().between(min_interval, max_interval).all()
    }).reset_index()

    # Merge with trip_counts to include Row_Count
    trip_validity = trip_validity.merge(trip_counts, on=trip_column)

    # Define validity based on both time differences and row count
    trip_validity['IsValid'] = trip_validity.apply(
        lambda row: row['Time_Diff'] and row['Row_Count'] > min_rows, axis=1
    )

    # Filter trips that are valid
    valid_trips = trip_validity[trip_validity['IsValid']][trip_column].tolist()

    return valid_trips



def filter_report(report_df, valid_trips, trip_column='report_trip_number'):
    """
    Filters the Report DataFrame to include only valid trips.

    Parameters:
        report_df (pd.DataFrame): DataFrame containing trip summaries.
        valid_trips (List): List of valid trip identifiers.
        trip_column (str): Column name in the Report sheet that contains trip identifiers.

    Returns:
        Filtered DataFrame with only valid trips.
    """
    if trip_column not in report_df.columns:
        raise ValueError(f"Report sheet must contain '{trip_column}' column.")

    # Split the report_trip_number by '_' and take the second part as trip number
    # Example: 'trip_1' -> 1
    def extract_trip_number(report_trip):
        try:
            parts = report_trip.split('_')
            if len(parts) < 2:
                raise ValueError(f"Invalid format: '{report_trip}'")
            trip_num = int(parts[1])
            return trip_num
        except (IndexError, ValueError) as e:
            print(f"Warning: Could not extract trip number from '{report_trip}'. Skipping this entry.")
            return pd.NA

    report_df['trip_numeric'] = report_df[trip_column].apply(extract_trip_number)

    # Identify and report rows with NaN trip_numeric
    nan_trips = report_df[report_df['trip_numeric'].isna()]
    if not nan_trips.empty:
        print(f"Warning: {len(nan_trips)} trip(s) in Report sheet have invalid format and will be skipped.")

    # Filter based on valid_trips and ensure trip_numeric is not NaN
    filtered_report = report_df[
        report_df['trip_numeric'].isin(valid_trips) & report_df['trip_numeric'].notna()
        ].reset_index(drop=True)

    return filtered_report


def create_boxplots(report_df, trip_column='report_trip_number', output_dir=None):
    """
    Creates boxplots for each numerical column in the Report DataFrame.

    Parameters:
        report_df (pd.DataFrame): Filtered Report DataFrame with valid trips.
        trip_column (str): Column name that contains trip identifiers.
        output_dir (str, optional): Directory to save the plots. If None, plots are shown.
    """
    # Identify numerical columns (excluding the trip identifier column)
    numerical_cols = report_df.select_dtypes(include=['number']).columns.tolist()

    # Remove the numeric trip identifier column if present
    if 'trip_numeric' in numerical_cols:
        numerical_cols.remove('trip_numeric')

    for col in numerical_cols:
        plt.figure(figsize=(8, 6))
        sns.boxplot(y=report_df[col])
        plt.title(f'Boxplot of {col} for Valid Trips')
        plt.ylabel(col)
        plt.tight_layout()

        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            plot_path = os.path.join(output_dir, f"boxplot_{col}.png")
            plt.savefig(plot_path)
            plt.close()
            print(f"Boxplot for '{col}' saved to '{plot_path}'")
        else:
            plt.show()


def calculate_slope(trip_data, altitude_col='altitude', mileage_col='Cumulative_mileage', chunk_size=100):
    """
    Calculates the slope for each set of 100 consecutive rows in trip_data and assigns it to a new 'slope' column.

    Parameters:
        trip_data (pd.DataFrame): DataFrame containing data for a single trip.
        altitude_col (str): Name of the column representing altitude.
        mileage_col (str): Name of the column representing cumulative mileage.
        chunk_size (int): Number of rows in each chunk to calculate slope.

    Returns:
        pd.DataFrame: DataFrame with an additional 'slope' column.
    """
    # Initialize 'slope' column with NaN
    trip_data['slope'] = pd.NA

    total_rows = len(trip_data)

    # Iterate over the DataFrame in chunks of 100 rows
    for i in range(0, total_rows - chunk_size, chunk_size):
        start_altitude = trip_data.iloc[i][altitude_col]
        end_altitude = trip_data.iloc[i + chunk_size][altitude_col]
        start_mileage = trip_data.iloc[i][mileage_col]
        end_mileage = trip_data.iloc[i + chunk_size][mileage_col]

        mileage_diff = end_mileage - start_mileage

        if mileage_diff != 0:
            slope = (end_altitude - start_altitude) / mileage_diff / 10
            trip_data.loc[i:i + chunk_size - 1, 'slope'] = slope
        else:
            # If mileage_diff is zero, assign NaN
            trip_data.loc[i:i + chunk_size - 1, 'slope'] = pd.NA

    return trip_data


def save_valid_trips_as_csv(data_df, valid_trips, output_dir, trip_column='trip', altitude_col='altitude',
                            mileage_col='Cumulative_mileage'):
    """
    Saves each valid trip's data as a separate CSV file, adding a 'slope' column.

    Parameters:
        data_df (pd.DataFrame): Original Data DataFrame containing all trips.
        valid_trips (List): List of valid trip identifiers.
        output_dir (str): Directory to save the CSV files.
        trip_column (str): Column name that contains trip identifiers.
        altitude_col (str): Column name for altitude.
        mileage_col (str): Column name for cumulative mileage.
    """
    if trip_column not in data_df.columns:
        raise ValueError(f"Data sheet must contain '{trip_column}' column.")

    os.makedirs(output_dir, exist_ok=True)

    for trip in valid_trips:
        trip_data = data_df[data_df[trip_column] == trip].reset_index(drop=True)

        if trip_data.empty:
            print(f"Warning: No data found for valid trip '{trip}'. Skipping CSV export.")
            continue

        # Check if 'altitude' and 'Cumulative_mileage' columns exist
        if altitude_col not in trip_data.columns or mileage_col not in trip_data.columns:
            print(
                f"Warning: Trip '{trip}' missing '{altitude_col}' or '{mileage_col}' columns. Skipping slope calculation.")
            trip_data['slope'] = pd.NA
        else:
            # Calculate and assign 'slope' column
            trip_data = calculate_slope(trip_data, altitude_col, mileage_col, chunk_size=100)

        # Sanitize trip identifier for filename (remove or replace problematic characters)
        trip_safe = str(trip).replace('/', '_').replace('\\', '_').replace(':', '_').replace('*', '_') \
            .replace('?', '_').replace('"', '_').replace('<', '_').replace('>', '_') \
            .replace('|', '_')
        filename = f"Trip_{trip_safe}.csv"
        filepath = os.path.join(output_dir, filename)

        # Prevent overwriting existing files by appending a counter if necessary
        counter = 1
        base_filename, ext = os.path.splitext(filename)
        while os.path.exists(filepath):
            filepath = os.path.join(output_dir, f"{base_filename}_{counter}{ext}")
            counter += 1

        trip_data.to_csv(filepath, index=False)
        print(f"Valid trip '{trip}' saved to '{filepath}'")


def create_summary_report(report_df, trip_column='report_trip_number', output_dir=None):
    """
    Optional: Create a summary CSV of valid trips from the Report sheet.

    Parameters:
        report_df (pd.DataFrame): Filtered Report DataFrame with valid trips.
        trip_column (str): Column name that contains trip identifiers.
        output_dir (str, optional): Directory to save the summary CSV. If None, does not save.
    """
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        summary_path = os.path.join(output_dir, "filtered_report_summary.csv")
        report_df.to_csv(summary_path, index=False)
        print(f"Filtered Report summary saved to '{summary_path}'")


def main():
    # Path to your Excel file
    excel_file = '/content/drive/MyDrive/Qani_datas.xlsx'  # Replace with your actual file path
    excel_files = 'combined_slope_added.csv'
    # Sheet names
    data_sheet = 'Data'
    report_sheet = 'Report'

    # Column names
    data_trip_column = 'trip'
    data_time_column = 'time'
    report_trip_column = 'report_trip_number'

    # Columns required for slope calculation
    altitude_col = 'altitude'
    mileage_col = 'Cumulative_mileage'

    # Output directories
    plots_output_dir = 'boxplots'  # Directory to save boxplots
    csv_output_dir = 'valid_trips_csv'  # Directory to save valid trip CSV files
    summary_output_dir = 'summary_reports'  # Directory to save summary reports (optional)

    # Read the Excel file
    try:
        # data_df = pd.read_excel(excel_file, sheet_name=data_sheet)
        data_df = pd.read_csv(excel_files)

        report_df = pd.read_excel(excel_file, sheet_name=report_sheet)
    except FileNotFoundError:
        print(f"Error: The file '{excel_file}' was not found.")
        return
    except ValueError as e:
        print(f"Error: {e}")
        return

    # Check if required columns exist in Data sheet
    if data_trip_column not in data_df.columns or data_time_column not in data_df.columns:
        print(f"Error: '{data_sheet}' sheet must contain '{data_trip_column}' and '{data_time_column}' columns.")
        return

    # Check if required column exists in Report sheet
    if report_trip_column not in report_df.columns:
        print(f"Error: '{report_sheet}' sheet must contain '{report_trip_column}' column.")
        return

    # Identify valid trips
    try:
        valid_trips = identify_valid_trips(
            data_df,
            trip_column=data_trip_column,
            time_column=data_time_column,
            min_interval=400,
            max_interval=2000,
            min_rows=600  # New condition: minimum number of rows
        )
    except ValueError as ve:
        print(f"Error during trip validation: {ve}")
        return

    print(f"Number of valid trips: {len(valid_trips)}")

    if not valid_trips:
        print("No valid trips found based on the given time interval and row count conditions.")
        return

    # Filter the Report sheet
    try:
        filtered_report = filter_report(
            report_df,
            valid_trips,
            trip_column=report_trip_column
        )
    except ValueError as ve:
        print(f"Error during Report sheet filtering: {ve}")
        return

    print(f"Filtered Report has {filtered_report.shape[0]} rows.")

    if filtered_report.empty:
        print("No matching trips found in the Report sheet for the valid trips identified.")
        return

    # Create boxplots
    create_boxplots(
        filtered_report,
        trip_column=report_trip_column,
        output_dir=plots_output_dir
    )

    # Save each valid trip as a separate CSV file, including the 'slope' column
    save_valid_trips_as_csv(
        data_df,
        valid_trips,
        output_dir=csv_output_dir,
        trip_column=data_trip_column,
        altitude_col=altitude_col,
        mileage_col=mileage_col
    )

    # Optional: Save a summary of the filtered Report sheet
    create_summary_report(
        filtered_report,
        trip_column=report_trip_column,
        output_dir=summary_output_dir
    )

    print("Processing complete.")


if __name__ == "__main__":
    main()


Number of valid trips: 80
Filtered Report has 80 rows.


/usr/local/lib/python3.10/dist-packages/seaborn/categorical.py:640: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)


Boxplot for 'data_count' saved to 'boxplots/boxplot_data_count.png'


/usr/local/lib/python3.10/dist-packages/seaborn/categorical.py:640: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)


Boxplot for 'trip_time' saved to 'boxplots/boxplot_trip_time.png'
Boxplot for 'trip_distance' saved to 'boxplots/boxplot_trip_distance.png'


...

Boxplot for 'trip_coolant_temperature_start' saved to 'boxplots/boxplot_trip_coolant_temperature_start.png'


/usr/local/lib/python3.10/dist-packages/seaborn/categorical.py:640: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)


Boxplot for 'trip_fuel_consumption_lit' saved to 'boxplots/boxplot_trip_fuel_consumption_lit.png'


/usr/local/lib/python3.10/dist-packages/seaborn/categorical.py:640: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)


Boxplot for 'trip_fuel_consumption_avg' saved to 'boxplots/boxplot_trip_fuel_consumption_avg.png'
Valid trip '4' saved to 'valid_trips_csv/Trip_4.csv'
Valid trip '6' saved to 'valid_trips_csv/Trip_6.csv'
Valid trip '8' saved to 'valid_trips_csv/Trip_8.csv'
Valid trip '13' saved to 'valid_trips_csv/Trip_13.csv'
Valid trip '21' saved to 'valid_trips_csv/Trip_21.csv'
Valid trip '27' saved to 'valid_trips_csv/Trip_27.csv'
Valid trip '30' saved to 'valid_trips_csv/Trip_30.csv'
Valid trip '32' saved to 'valid_trips_csv/Trip_32.csv'
Valid trip '33' saved to 'valid_trips_csv/Trip_33.csv'
Valid trip '35' saved to 'valid_trips_csv/Trip_35.csv'
Valid trip '44' saved to 'valid_trips_csv/Trip_44.csv'
Valid trip '50' saved to 'valid_trips_csv/Trip_50.csv'
Valid trip '53' saved to 'valid_trips_csv/Trip_53.csv'
Valid trip '57' saved to 'valid_trips_csv/Trip_57.csv'
Valid trip '58' saved to 'valid_trips_csv/Trip_58.csv'
Valid trip '63' saved to 'valid_trips_csv/Trip_63.csv'
Valid trip '68' saved to 'va

# **Devide to 600Rows**

In [19]:
import pandas as pd
import os

def split_csv_file(input_file, output_file_prefix, rows_per_file):
    df = pd.read_csv(input_file)

    total_rows = len(df)
    num_files = (total_rows // rows_per_file) + (1 if total_rows % rows_per_file != 0 else 0)

    for i in range(num_files):
        start_row = i * rows_per_file
        end_row = start_row + rows_per_file

        # Create a subset DataFrame
        subset_df = df.iloc[start_row:end_row]

        # Generate output file name
        output_file = f"{output_file_prefix}_{i + 1}.csv"

        # Save the subset DataFrame to a new CSV file
        subset_df.to_csv(output_file, index=False)

    print(f"{input_file} split into {num_files} files, including the last segment if under {rows_per_file} rows.")

# Path to the folder containing CSV files
folder_path = '/content/valid_trips_csv'
folder_path1 = '/content/valid_trips_csv_600row'
# Process each CSV file in the specified folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        input_file_path = os.path.join(folder_path, filename)
        output_file_prefix = os.path.join(folder_path1, filename[:-4])  # Remove .csv for output file naming
        rows_per_file = 600

        split_csv_file(input_file_path, output_file_prefix, rows_per_file)

print("All files processed successfully.")


/content/valid_trips_csv/Trip_141.csv split into 5 files, including the last segment if under 600 rows.
/content/valid_trips_csv/Trip_184.csv split into 3 files, including the last segment if under 600 rows.
/content/valid_trips_csv/Trip_166.csv split into 6 files, including the last segment if under 600 rows.
/content/valid_trips_csv/Trip_121.csv split into 2 files, including the last segment if under 600 rows.
/content/valid_trips_csv/Trip_97.csv split into 4 files, including the last segment if under 600 rows.
/content/valid_trips_csv/Trip_155.csv split into 4 files, including the last segment if under 600 rows.
/content/valid_trips_csv/Trip_125.csv split into 4 files, including the last segment if under 600 rows.
/content/valid_trips_csv/Trip_217.csv split into 4 files, including the last segment if under 600 rows.
/content/valid_trips_csv/Trip_21.csv split into 3 files, including the last segment if under 600 rows.
/content/valid_trips_csv/Trip_120.csv split into 2 files, includin

In [20]:
!zip -r valid_trips_csv_600row.zip valid_trips_csv_600row

  adding: valid_trips_csv_600row/ (stored 0%)
  adding: valid_trips_csv_600row/Trip_134_4.csv (deflated 83%)
  adding: valid_trips_csv_600row/Trip_116_3.csv (deflated 83%)
  adding: valid_trips_csv_600row/Trip_186_2.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_186_5.csv (deflated 84%)
  adding: valid_trips_csv_600row/Trip_137_3.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_90_2.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_137_2.csv (deflated 84%)
  adding: valid_trips_csv_600row/Trip_180_2.csv (deflated 80%)
  adding: valid_trips_csv_600row/Trip_186_3.csv (deflated 87%)
  adding: valid_trips_csv_600row/Trip_79_2.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_198_2.csv (deflated 87%)
  adding: valid_trips_csv_600row/Trip_172_1.csv (deflated 82%)
  adding: valid_trips_csv_600row/Trip_102_3.csv (deflated 81%)
  adding: valid_trips_csv_600row/Trip_141_1.csv (deflated 80%)
  adding: valid_trips_csv_600row/Trip_130_3.csv (deflated 82%)
  adding: v

# **Augment V7**

In [25]:
import os

def count_csv_files(folder_path):
    # List all files in the specified folder
    files = os.listdir(folder_path)

    # Filter for CSV files
    csv_files = [file for file in files if file.endswith('.csv')]

    # Count the number of CSV files
    count = len(csv_files)

    print(f"Number of CSV files in '{folder_path}': {count}")
    return count

# Path to the folder containing CSV files
folder_path = '/content/data_aug(3_slices_with_repeated)_cluster_5'

# Call the function to count CSV files
count_csv_files(folder_path)

Number of CSV files in '/content/data_aug(3_slices_with_repeated)_cluster_5': 49690


49690

In [22]:
import pandas as pd
import os
import itertools

def adjust_sequence(prev_value, slice_values):
    adjusted_values = [prev_value]
    for i in range(len(slice_values)):
        delta = slice_values[i] - slice_values[i - 1] if i > 0 else 0
        adjusted_values.append(adjusted_values[-1] + delta)
    return adjusted_values[1:]

# def process_combination(slices):
#     processed_data = pd.DataFrame()
#     num_slices = len(slices)
#     for i in range(num_slices):
#         if i > 0:
#             for column in ['Time', 'Cumulative mileage', 'Trip fuel consumption']:
#                 prev_value = processed_data[column].iloc[-1]
#                 slices[i][column] = adjust_sequence(prev_value, list(slices[i][column]))
#             slices[i] = slices[i].iloc[1:]
#         processed_data = pd.concat([processed_data, slices[i]])
#     return processed_data


def process_combination(slices):
    processed_data = pd.DataFrame()
    num_slices = len(slices)
    for i in range(num_slices):
        if i > 0:
            for column in ['Time', 'Cumulative mileage', 'Trip fuel consumption']:
                try:
                    prev_value = processed_data[column].iloc[-1]
                except IndexError:
                    # Handle empty processed_data: You could either skip this part or use a default value
                    prev_value = slices[i][column].iloc[0]  # Using first value of current slice as fallback
                    print(
                        f"Warning: processed_data for column '{column}' is empty. Using first value from current slice.")

                slices[i][column] = adjust_sequence(prev_value, list(slices[i][column]))
            slices[i] = slices[i].iloc[1:]  # Skip first row of the current slice (as it is already adjusted)
        processed_data = pd.concat([processed_data, slices[i]])
    return processed_data

num_slices = 2


source_folder = '/content/valid_trips_csv_600row'
target_folder = 'data_aug(3_slices_with_repeated)_cluster_5'

os.makedirs(target_folder, exist_ok=True)

driver_files = {}
for filename in os.listdir(source_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(source_folder, filename)
        df = pd.read_csv(file_path)
        # comment this for offline data loggers
        df.rename(columns={'Engine_speed': 'Engine speed',
                           'time': 'Time',
                           'Vehicle_Speed': 'Speed',
                           'Trip_fuel_consumption': 'Trip fuel consumption',
                           'Throttle_position': 'Throttle position',
                           'Accelerator_pedal_position': 'Accelerator pedal position',
                           'Cumulative_mileage': 'Cumulative mileage'
                           }, inplace=True)

        min_mileage = df['Cumulative mileage'].min()
        max_mileage = df['Cumulative mileage'].max()
        slice_size = (max_mileage - min_mileage) / num_slices

        # driver_name = driver_data.loc[driver_data['fileName'].str.contains(filename[:-4], na=False), 'driver_name_english'].values[0]
        driver_name = filename[:-4]
        mileage_slices = [df[(df['Cumulative mileage'] >= min_mileage + i * slice_size) &
                             (df['Cumulative mileage'] < min_mileage + (i + 1) * slice_size)]
                          for i in range(num_slices)]

        mileage_slices[-1] = pd.concat([mileage_slices[-1], df[df['Cumulative mileage'] == max_mileage]])
        driver_files[driver_name] = mileage_slices

driver_combinations = list(itertools.product(driver_files.keys(), repeat=num_slices))

file_count = 0
# for combination in driver_combinations:
#     slices = [driver_files[driver][slice_index] for driver, slice_index in zip(combination, range(num_slices))]
#     combined_data = process_combination(slices)
#     output_file = os.path.join(target_folder,
#                                f"{'_'.join([f'{driver}(slice_{slice_index + 1})' for driver, slice_index in zip(combination, range(num_slices))])}.csv")
#     combined_data.to_csv(output_file, index=False)
#     file_count += 1

for combination in driver_combinations:
    slices = [driver_files[driver][slice_index] for driver, slice_index in zip(combination, range(num_slices))]
    combined_data = process_combination(slices)

    # Only save if the length of the combined DataFrame is 600 or more
    if len(combined_data) >= 600:
        output_file = os.path.join(target_folder,
                                   f"{'_'.join([f'{driver}(slice_{slice_index + 1})' for driver, slice_index in zip(combination, range(num_slices))])}.csv")
        combined_data.to_csv(output_file, index=False)
        file_count += 1

Streaming output truncated to the last 5000 lines.


In [24]:
!zip -r 'data_aug(3_slices_with_repeated)_qani.zip' 'data_aug(3_slices_with_repeated)_cluster_5'

Streaming output truncated to the last 5000 lines.
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_90_13(slice_1)_Trip_58_9(slice_2).csv (deflated 86%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_155_3(slice_1)_Trip_57_23(slice_2).csv (deflated 82%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_185_1(slice_1)_Trip_6_1(slice_2).csv (deflated 85%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_79_2(slice_1)_Trip_27_4(slice_2).csv (deflated 81%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_155_1(slice_1)_Trip_79_4(slice_2).csv (deflated 84%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_164_1(slice_1)_Trip_13_6(slice_2).csv (deflated 86%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_32_1(slice_1)_Trip_90_38(slice_2).csv (deflated 83%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_57_14(slice_1)_Trip_90_25(slice_2).csv (deflated 81%)
  adding: data_aug(3_slices_with_repeated)_cluster_5/Trip_125